In [ ]:
!pip install -q kaggle
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"simonececchinato","key":"aa3d76c71079684e73aaa4e257a6898b"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                         480KB  2023-01-04 12:50:31           8779        300  1.0              
googleai/musiccaps                                              MusicCaps                                          793KB  2023-01-25 09:25:48           1705        177  0.9411765        
themrityunjaypathak/most-subscribed-1000-youtube-channels       Most Subscribed 1000 Youtube Channels               28KB  2023-01-21 14:42:05           2177         72  1.0              
nitishsharma01/olympics-124-years-datasettill-2020              O

In [ ]:
!kaggle datasets download -d piyushkumar18/animal-image-classification-dataset

 99% 1.45G/1.47G [00:14<00:00, 166MB/s]
100% 1.47G/1.47G [00:14<00:00, 111MB/s]


In [ ]:
!mkdir /content/animal_data
!unzip -qq /content/animal-image-classification-dataset.zip -d /content/animal_data/

In [ ]:
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb

import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.preprocessing import LabelEncoder

import cv2


## Building training set and validation set


### Selecting the images from the entire dataset

In [ ]:
files.upload();

Saving test_animals.txt to test_animals.txt


In [ ]:
# path_noadd = path of the txt file with the paths of the images not to add
# return the dataframe

def build_dataset(path_noadd):

  #starting path for the kaggle dataset
  start_path = '/content/animal_data/Animal Image Dataset/'

  with open(path_noadd) as file:
    val_paths = [line.rstrip() for line in file]

  import os

  images = []
  labels = []

  for folders, subfolders, files in os.walk(start_path,topdown=True):
    label = folders.split('/')[4]
    for file in files:

      path_file = start_path + label + '/' + file

      if path_file not in val_paths:  
        images.append(path_file)
        labels.append(label)
      
  data = {'Images':images, 'Labels':labels} 
  data = pd.DataFrame(data) 

  lb = LabelEncoder()
  data['encoded_labels'] = lb.fit_transform(data['Labels'])

  return data

In [ ]:
data = build_dataset(path_noadd = "test_animals.txt")
print(len(data))

14778


In [ ]:
batch_size = 128
train_dim = 12800
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(data)
indices = list(range(dataset_size))
#split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[:train_dim], indices[train_dim:]

print(f"train dataset length = {len(train_indices)}")
print(f"validation dataset length = {len(val_indices)}")

train dataset length = 12800
validation dataset length = 1978


In [ ]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
transform = transforms.Compose([
                  transforms.Resize((224,224)),
                  transforms.ToTensor(),
                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                  ])

In [ ]:
class Animals_Dataset(Dataset):
    def __init__(self, img_data,transform=None):
        self.transform = transform
        self.img_data = img_data
        
    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        img_name = self.img_data.loc[index, 'Images']
        image = Image.open(img_name)
        image = image.convert('RGB')

        gray = image.convert('L')
        gray_image = gray.convert('RGB')
        
        label = torch.tensor(self.img_data.loc[index, 'encoded_labels'])
        
        if self.transform is not None:
            image = self.transform(image)
            gray_image = self.transform(gray_image)

        return image, gray_image, label

In [ ]:
animal_dataset = Animals_Dataset(data,transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(animal_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(animal_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


##Visualization


In [ ]:
def img_display(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

In [ ]:
# get some random training images
#dataiter = iter(train_loader)
#images, labels = dataiter.next()
images,_, labels = next(iter(validation_loader))
animal_types = {0: 'butterfly', 1: 'cat', 2: 'cow', 3: 'dog', 4: 'elephant', 5: 'hen', 6: 'horse', 7: 'mokey', 8: 'panda', 9: 'sheep', 10: 'spider', 11: 'squirrel'}

# Viewing data examples used for training
fig, axis = plt.subplots(3, 5, figsize=(10, 15))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label = images[i], labels[i]
        ax.imshow(img_display(image)) # add image
        ax.set(title = f"{animal_types[label.item()]}") # add label

## Color Image Classification

In [ ]:
# Device configuration
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

Training device: cuda


In [ ]:
from torchvision import models, transforms

color_model = models.vgg16(pretrained=True);

color_model.classifier[6] = nn.Linear(in_features=4096, out_features=12);

color_model.to(device);
color_model.train();


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
learning_rate = 0.005

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(color_model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

# Train the model
total_step = len(train_loader)

## Training

In [ ]:
n_epochs = 1
print_every = 25
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')

    for i, (images,_, labels) in tqdm(enumerate(train_loader), total = len(train_loader)):

        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = color_model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)

        if (i) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, i, total_step, loss.item()))
            
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')


    batch_loss = 0
    total_t=0
    correct_t=0


    #validation
    with torch.no_grad():
        color_model.eval()

        for (images,_,labels) in tqdm(validation_loader,total = len(validation_loader)):

          #Move tensors to the configured device
          images = images.to(device)
          labels = labels.to(device)

          outputs = color_model(images)
          loss = criterion(outputs, labels)

          batch_loss += loss.item()
          _,pred_t = torch.max(outputs, dim=1)
          correct_t += torch.sum(pred_t==labels).item()
          total_t += labels.size(0)

        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}\n')


        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(color_model.state_dict(), 'vgg16-color.pt')
            print('Detected network improvement, saving current model')

    color_model.train()

Epoch 1



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/1], Step [0/100], Loss: 0.0891
Epoch [1/1], Step [25/100], Loss: 0.0563
Epoch [1/1], Step [50/100], Loss: 0.0146
Epoch [1/1], Step [75/100], Loss: 0.0235

train loss: 0.0491, train acc: 98.4922


  0%|          | 0/16 [00:00<?, ?it/s]

validation loss: 0.1229, validation acc: 96.3094

Detected network improvement, saving current model


In [ ]:
# mount it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy it there
!cp /content/vgg16-color.pt /content/drive/MyDrive/TrainedNets

# Grayscale Image Classification Training

In [ ]:

gray_model = models.vgg16(pretrained=True)

gray_model.classifier[6] = nn.Linear(in_features=4096, out_features=12)

gray_model.to(device)
gray_model.train()


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
learning_rate = 0.005

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(gray_model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

# Train the model
total_step = len(train_loader)

In [ ]:
n_epochs = 3
print_every = 25
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')

    for i, ( _, gray_img, labels) in tqdm(enumerate(train_loader), total = len(train_loader)):

        # Move tensors to the configured device
        gray_img = gray_img.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = gray_model(gray_img)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)

        if (i) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, i, total_step, loss.item()))
            
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')


    batch_loss = 0
    total_t=0
    correct_t=0


    #validation
    with torch.no_grad():
        gray_model.eval()

        for ( _, gray_img ,labels) in tqdm(validation_loader,total = len(validation_loader)):

          #Move tensors to the configured device
          gray_img = gray_img.to(device)
          labels = labels.to(device)

          outputs = gray_model(gray_img)
          loss = criterion(outputs, labels)

          batch_loss += loss.item()
          _,pred_t = torch.max(outputs, dim=1)
          correct_t += torch.sum(pred_t==labels).item()
          total_t += labels.size(0)

        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}\n')


        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(gray_model.state_dict(), 'vgg16-gray.pt')
            print('Detected network improvement, saving current model')

    gray_model.train()

Epoch 1



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/3], Step [0/100], Loss: 2.5896
Epoch [1/3], Step [25/100], Loss: 0.3645
Epoch [1/3], Step [50/100], Loss: 0.3354
Epoch [1/3], Step [75/100], Loss: 0.1140

train loss: 0.4038, train acc: 86.9844


  0%|          | 0/16 [00:00<?, ?it/s]

validation loss: 0.2027, validation acc: 93.0738

Detected network improvement, saving current model
Epoch 2



...

validation loss: 0.1963, validation acc: 93.9838

Detected network improvement, saving current model
Epoch 3



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [3/3], Step [0/100], Loss: 0.0663
Epoch [3/3], Step [25/100], Loss: 0.0503
Epoch [3/3], Step [50/100], Loss: 0.0337
Epoch [3/3], Step [75/100], Loss: 0.0600

train loss: 0.1974, train acc: 98.0156


  0%|          | 0/16 [00:00<?, ?it/s]

validation loss: 0.1844, validation acc: 95.1466

Detected network improvement, saving current model


In [ ]:
!cp /content/vgg16-gray.pt /content/drive/MyDrive/TrainedNets